# Acton Station NFDRS Test Dashboard

In [ ]:

import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# -- Set up fixed values for Acton --
station_id = "45438"
station_name = "Acton (45438)"
index_dropdown = widgets.Dropdown(
    options={
        "Burning Index (BI)": "burningIndex",
        "Energy Release Component (ERC)": "energyReleaseComponent",
        "Ignition Component (IC)": "ignitionComponent",
        "Spread Component (SC)": "spreadComponent",
        "KBDI": "KBDI",
        "1000hr Dead FM": "thousandHR_TL_FuelMoisture"
    },
    description="Index"
)

plot_button = widgets.Button(description="Generate Chart", button_style='primary')

def get_df(url, component):
    try:
        r = requests.get(url)
        if r.ok:
            df = pd.read_csv(StringIO(r.text))
            df.columns = [c.lower() for c in df.columns]
            if 'observation_time' in df.columns and component in df.columns:
                df['datetime'] = pd.to_datetime(df['observation_time'], errors='coerce')
                df = df.dropna(subset=['datetime'])
                return df[['datetime', component]]
    except Exception as e:
        print("Fetch error:", e)
    return pd.DataFrame()

def plot_chart(button):
    output.clear_output()
    with output:
        component = index_dropdown.value
        print(f"Component: {component} for Station: {station_name}")

        today = datetime.utcnow()
        year = today.year
        time_ranges = {
            "Historical": ("2009-01-01T00:30:00Z", f"{year-1}-12-31T23:30:00Z"),
            "Current": (f"{year}-01-01T00:30:00Z", today.strftime("%Y-%m-%dT00:30:00Z")),
            "Forecast": (today.strftime("%Y-%m-%dT00:30:00Z"), (today + timedelta(days=7)).strftime("%Y-%m-%dT00:30:00Z"))
        }

        fig = go.Figure()
        line_styles = {"Historical": "dot", "Current": "solid", "Forecast": "dash"}
        data_found = False

        for label, (start, end) in time_ranges.items():
            dataset = "forecast" if label == "Forecast" else "all"
            url = f"https://fems.fs2c.usda.gov/api/climatology/download-nfdr?stationIds={station_id}&startDate={start}&endDate={end}&dataFormat=csv&dataset={dataset}&fuelModels=Y"
            df = get_df(url, component)
            if not df.empty:
                data_found = True
                fig.add_trace(go.Scatter(
                    x=df['datetime'],
                    y=df[component],
                    mode='lines',
                    name=label,
                    line=dict(dash=line_styles[label])
                ))

        if not data_found:
            print("⚠️ No data returned for this station/component.")
            return

        fig.update_layout(
            title=f"{station_name} – {component}",
            xaxis_title="Date",
            yaxis_title=component,
            hovermode="x unified",
            template="plotly_white"
        )
        fig.show()

plot_button.on_click(plot_chart)

output = widgets.Output()
ui = widgets.VBox([index_dropdown, plot_button, output])
display(ui)
